In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pycryptodome cryptography pyopenssl --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
  Attempting uninstall: pyopenssl
    Found existing installation: pyOpenSSL 24.2.1
    Uninstalling pyOpenSSL-24.2.1:
      Successfully uninstalled pyOpenSSL-24.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2 1.21.3 requires pyOpenSSL<=24.2.1,>=19.1.0, but you have pyopenssl 25.3.0 which is incompatible.


In [3]:
# Перевірка версій
from Crypto.PublicKey import RSA
import cryptography
import OpenSSL

In [5]:
import os
import timeit

# Тестові дані
plaintext = "Це секретне повідомлення для гібридного шифрування.".encode('utf-8') + b"A" * 100000

# Функція для повторюваних тестів
def benchmark(func, number=10):
    return timeit.timeit(func, number=number) / number

print(f"Розмір повідомлення: {len(plaintext) / 1024:.1f} KB")

Розмір повідомлення: 97.8 KB


In [6]:
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Random import get_random_bytes
import base64

def hybrid_encrypt_pycryptodome(data: bytes):
    # Генерація RSA ключів (2048 біт)
    rsa_key = RSA.generate(2048)
    private_key = rsa_key.export_key()
    public_key = rsa_key.publickey().export_key()

    # Сесійний ключ для AES
    session_key = get_random_bytes(32)  # AES-256

    # Шифрування сесійного ключа RSA-OAEP
    rsa_cipher = PKCS1_OAEP.new(rsa_key.publickey())
    enc_session_key = rsa_cipher.encrypt(session_key)

    # Шифрування даних AES-GCM
    aes_cipher = AES.new(session_key, AES.MODE_GCM)
    ciphertext, tag = aes_cipher.encrypt_and_digest(data)

    # Повертаємо все для передачі
    return {
        "enc_session_key": base64.b64encode(enc_session_key).decode(),
        "nonce": base64.b64encode(aes_cipher.nonce).decode(),
        "tag": base64.b64encode(tag).decode(),
        "ciphertext": base64.b64encode(ciphertext).decode(),
        "private_key": private_key.decode()  # Для дешифрування
    }

def hybrid_decrypt_pycryptodome(enc_data: dict):
    private_key = RSA.import_key(enc_data["private_key"])
    enc_session_key = base64.b64decode(enc_data["enc_session_key"])
    nonce = base64.b64decode(enc_data["nonce"])
    tag = base64.b64decode(enc_data["tag"])
    ciphertext = base64.b64decode(enc_data["ciphertext"])

    rsa_cipher = PKCS1_OAEP.new(private_key)
    session_key = rsa_cipher.decrypt(enc_session_key)

    aes_cipher = AES.new(session_key, AES.MODE_GCM, nonce=nonce)
    plaintext = aes_cipher.decrypt_and_verify(ciphertext, tag)
    return plaintext

# Контрольний приклад
enc = hybrid_encrypt_pycryptodome(plaintext)
dec = hybrid_decrypt_pycryptodome(enc)
print("PyCryptodome: Успішно!" if dec == plaintext else "Помилка!")

PyCryptodome: Успішно!


In [7]:
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
import os

def hybrid_encrypt_cryptography(data: bytes):
    # Генерація RSA ключів
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    public_key = private_key.public_key()

    private_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )

    # Сесійний ключ
    session_key = AESGCM.generate_key(bit_length=256)

    # Шифрування сесійного ключа RSA-OAEP
    enc_session_key = public_key.encrypt(
        session_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    # Шифрування даних AES-GCM
    aesgcm = AESGCM(session_key)
    nonce = os.urandom(12)
    ciphertext = aesgcm.encrypt(nonce, data, associated_data=None)

    return {
        "enc_session_key": base64.b64encode(enc_session_key).decode(),
        "nonce": base64.b64encode(nonce).decode(),
        "ciphertext": base64.b64encode(ciphertext).decode(),
        "private_key": private_pem.decode()
    }

def hybrid_decrypt_cryptography(enc_data: dict):
    private_key = serialization.load_pem_private_key(
        enc_data["private_key"].encode(),
        password=None
    )

    enc_session_key = base64.b64decode(enc_data["enc_session_key"])
    nonce = base64.b64decode(enc_data["nonce"])
    ciphertext = base64.b64decode(enc_data["ciphertext"])

    session_key = private_key.decrypt(
        enc_session_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    aesgcm = AESGCM(session_key)
    plaintext = aesgcm.decrypt(nonce, ciphertext, associated_data=None)
    return plaintext

# Контрольний приклад
enc = hybrid_encrypt_cryptography(plaintext)
dec = hybrid_decrypt_cryptography(enc)
print("cryptography (OpenSSL): Успішно!" if dec == plaintext else "Помилка!")

cryptography (OpenSSL): Успішно!


In [8]:
# Тести (10 повторів, середній час)
print("Бенчмарки (середній час на цикл, секунди):")

time_pycrypto = benchmark(lambda: hybrid_decrypt_pycryptodome(hybrid_encrypt_pycryptodome(plaintext)))
print(f"PyCryptodome: {time_pycrypto:.4f} с")

time_crypto = benchmark(lambda: hybrid_decrypt_cryptography(hybrid_encrypt_cryptography(plaintext)))
print(f"cryptography (OpenSSL bindings): {time_crypto:.4f} с")

Бенчмарки (середній час на цикл, секунди):
PyCryptodome: 0.5103 с
cryptography (OpenSSL bindings): 0.1135 с


In [28]:
file_path = '/content/drive/MyDrive/11semestr/mrkm/lab1/test_text_cr1.txt'

In [29]:
# Читаємо як бінарні дані
with open(file_path, 'rb') as f:
    plaintext_bytes = f.read()

# Декодуємо в текст (українська — UTF-8)
try:
    plaintext_text = plaintext_bytes.decode('utf-8')
except UnicodeDecodeError:
    plaintext_text = plaintext_bytes.decode('utf-8', errors='replace')

print(f"Розмір файлу: {len(plaintext_bytes) / 1024:.1f} КБ")
print("\nПерші 200 символів тексту:")
print(plaintext_text[:200])

print("\nОстанні 200 символів тексту:")
print(plaintext_text[-200:])

Розмір файлу: 214.5 КБ

Перші 200 символів тексту:
По весні 1663 року двоє подорожніх, верхи на добрих конях, ізближались до Києва з Білогородського шляху. Один був молодий собі козак, збройний, як до війни; другий, по одежі і по сивій бороді, сказать

Останні 200 символів тексту:
о діло!

— О боже, спасителю! Се наші їдуть! — закричала Леся, глянувши на дорогу. А то стояла все, мов нежива, коло чорногорця, дивлячись на страшне одноборство.

Справді, по полю мчались козаки.


In [30]:
print("\nНові бенчмарки (середній час на повний цикл шифрування + дешифрування):")

time_pycrypto = benchmark(lambda: hybrid_decrypt_pycryptodome(hybrid_encrypt_pycryptodome(plaintext)))
print(f"PyCryptodome: {time_pycrypto:.4f} с")

time_crypto = benchmark(lambda: hybrid_decrypt_cryptography(hybrid_encrypt_cryptography(plaintext)))
print(f"cryptography (OpenSSL bindings): {time_crypto:.4f} с")


Нові бенчмарки (середній час на повний цикл шифрування + дешифрування):
PyCryptodome: 0.6955 с
cryptography (OpenSSL bindings): 0.1198 с


In [33]:
print("=== Демонстрація роботи гібридного шифрування ===\n")
plaintext = plaintext_bytes
print(f"Оригінальний розмір тексту: {len(plaintext) / 1024:.1f} КБ")
print("Перші 200 символів оригіналу:")
print(plaintext.decode('utf-8', errors='replace')[:200])
print("Останні 200 символів оригіналу:")
print(plaintext.decode('utf-8', errors='replace')[-200:])

print("\n--- Шифрування через PyCryptodome ---")
enc_pyc = hybrid_encrypt_pycryptodome(plaintext)

print("\nЗашифрований сесійний ключ (base64, перші 100 символів):")
print(enc_pyc["enc_session_key"][:100] + "...")

print("Nonce (base64):", enc_pyc["nonce"][:50] + "...")
print("Тег аутентифікації (base64):", enc_pyc["tag"][:50] + "...")
print("Зашифрований текст (base64, перші 200 символів):")
print(enc_pyc["ciphertext"][:200] + "...")

# Дешифрування
dec_pyc = hybrid_decrypt_pycryptodome(enc_pyc)

print("\nДешифрування PyCryptodome:")
if dec_pyc == plaintext:
    print("Дешифрований текст збігається з оригіналом")
else:
    print("Текст не збігається")

print("\n--- Шифрування через cryptography (OpenSSL) ---")
enc_crypto = hybrid_encrypt_cryptography(plaintext)

print("\nЗашифрований сесійний ключ (base64, перші 100 символів):")
print(enc_crypto["enc_session_key"][:100] + "...")

print("Nonce (base64):", enc_crypto["nonce"][:50] + "...")
print("Зашифрований текст (base64, перші 200 символів):")
print(enc_crypto["ciphertext"][:200] + "...")

# Дешифрування
dec_crypto = hybrid_decrypt_cryptography(enc_crypto)

print("\nДешифрування cryptography:")
if dec_crypto == plaintext:
    print("Дешифрований текст збігається з оригіналом")
else:
    print("Текст не збігається")


=== Демонстрація роботи гібридного шифрування ===

Оригінальний розмір тексту: 214.5 КБ
Перші 200 символів оригіналу:
По весні 1663 року двоє подорожніх, верхи на добрих конях, ізближались до Києва з Білогородського шляху. Один був молодий собі козак, збройний, як до війни; другий, по одежі і по сивій бороді, сказать
Останні 200 символів оригіналу:
о діло!

— О боже, спасителю! Се наші їдуть! — закричала Леся, глянувши на дорогу. А то стояла все, мов нежива, коло чорногорця, дивлячись на страшне одноборство.

Справді, по полю мчались козаки.

--- Шифрування через PyCryptodome ---

Зашифрований сесійний ключ (base64, перші 100 символів):
jkAtDbjfaYeGIhSGgB9CE4VnRLiPeELAs9XyRqUdqucmQrsc2W2i3HBKoorpS1VWxgN2TNVnFaXm7hypbE5vE/SYgAEyl2LzNgOa...
Nonce (base64): HWiSSP21lTQRJEwzkcUicw==...
Тег аутентифікації (base64): DbAKMsHPX4JacsHGtobvGg==...
Зашифрований текст (base64, перші 200 символів):
Xh5jhxwx9azRLrIciK0Tl0rkGx+t6uEMTCC3VIIMV4RFS8VPizW3VRefuXTcEzuVRXO7bIqpqcwKAFCRoQzs0IQzVLkWpGR8jPYtt